# Snakemake Workshop 

## Overview
This presentation is intended to give an overview of the workflow management tool, [Snakemake](https://snakemake.readthedocs.io/en/stable/). 

## Audience 
Examples in this presentation are targeted at energy system modellers who are comfortable working with Python. This workshop will use the [OSeMOSYS](http://www.osemosys.org/) Energy Modelling tool and [otoole](https://otoole.readthedocs.io/en/latest/) Python package quite heavily. With that said, there is no background knowledge needed in energy system modelling to follow this example. 

## Motivation 
"The Snakemake workflow management system is a tool to create reproducible and scalable data analyses." [ref](https://snakemake.readthedocs.io/en/stable/). Especially in the field of open-source energy modelling, ensuring data is reproducibale is often a requirment for publication. Introducting a tools to manage data pipelines not only simplifies your modelling workflow, but allows others to eaisly reporduce your results. 

## Similar tools
Snakemake is a Python implementation of a workflow management tool started from the field of bioinfomatics. Many other competitors exist, such as [nextflow](https://www.nextflow.io/), [airflow](https://github.com/apache/airflow), [galaxy](https://usegalaxy.org/), or [mage](https://www.mage.ai/), often with specific use cases or audiances. 

## References
Throughout this workshop, I will reference information from the following sources:
- [Snakemake homepage](https://snakemake.github.io/)
- [Snakemake paper](https://f1000research.com/articles/10-33)
- [Snakemake docs](https://snakemake.readthedocs.io/en/stable/index.html)
- [Reproduciable Data Analytic Workflows](https://lachlandeer.github.io/snakemake-econ-r-tutorial/)

# Background information 

## Common Terms 
- **Workflow**: A full data processing pipeline (a series of linked actions)
- **Rule**: Rules decompose the workflow into small steps (for examples, running a single script) 
- **Directed Acyclic Graph (DAC)**: Defines how rules link together 

## Rule Structure 
In general a rule is structured following the template: 

```python 
rule name: 
    input: 
        "input/file/path"
    output:
        "output/file/path"
    shell:
        "shell command"
```

Instead of `shell`, options also include to directly run python code through `run`
```python 
rule name: 
    input: 
        "input/file/path"
    output:
        "output/file/path"
    run:
        "print('hello world')"
```

Or to directly run scripts, if the correct directory structure is followed
```python 
rule name: 
    input: 
        "input/file/path"
    output:
        "output/file/path"
    script:
        "do_something.py"
```

### Other Notes
- Input and output variables are useable in the shell/run/script functions


# Workflow Overview 

Suppose our workflow consists of four steps: 
1. Run a series of Python scripts to generate annual demand data, capital cost data, and variable fuel cost data for different scenarios.
2. Convert all CSV data into a GNU MathProg datafile using otoole
3. Solve the model through GLPK
4. Visualize the total annual installed capacity

Within this workflow, the generation of all other parameter data is held constant. Each scenario you intend to run will depend on user defined parameters that affect your parameters described in step one (ie. increased electrifiction scenario, cheap solar scenario, ect...). A graphical overview of the workflow is given below. 

# Model Overview 
In this example, we have a sinple three technology OSeMOSYS model. It consists of a solar powerplant, a natural gas powerplant, and a hydro powerplant, which all generate electricity to meet an end-use demand. The model is given below. 



# Classical Workflow 
In many cases we will run each of these scenarios one-by-one, manually chnaging data in the script. We will first run through the steps needed to carry out the workflow. 

## 1. Create a Scenario Folder 
```bash 
mkdir scenarios 
```

## 2. Copy of the reference data 
```bash 
cp resources/data scenarios/data
```

## 3. Run the capital costs script
In this case we will implement a 1.5 scaling factor on solar panels

```bash 
python capital_costs.py scenarios/data/CapitalCosts.csv scenarios/data/CapitalCosts.csv SPV 1.5
```

## 4. Run the emission penalty script
In this case we will create a linear increase from 0 to 100 $/T

```bash 
python emission_penalty.py scenarios/data/EmissionsPenalty.csv scenarios/data/EmissionsPenalty.csv 0 100
```
## 5. Run the demand script
In this case we will scale the demand by 2

```bash 
python demand.py scenarios/data/SpecifiedAnnualDemand.csv scenarios/data/SpecifiedAnnualDemand.csv 2
```

## 6. Run the variable costs script
Note that this script must be run after the demand script 

```bash 
python variable_costs.py scenarios/data/SpecifiedAnnualDemand.csv scenarios/data/VariableCosts.csv scenarios/data/VariableCosts.csv
```

## 7. Use otoole to create a datafile 

```bash 
otoole convert csv datafile scenarios/data scenarios/data.txt resources/config.yaml
```

## 8. Create a folder to hold result data

```bash 
mkdir scenarios/results
```

## 9. Run GLPK to build the model 
OSeMOSYS requires the current working directory to have a `results`
folder when solving through GLPK, so we first change directories to 
the scenario folder 

```bash 
cd scenarios
glpsol -m ../resources/osemosys.txt -d scenarios/
cd ..
```

## 9. Run capacity plotting script 

```bash 
python plot_capacity.py scenarios/results/AnnualCapacity.csv scenarios/AnnualCapacity.png
```

## Issues
1. Running many scenarios becomes a nightmear 
2. Easy to make data mistakes (forgetting to run a script)
3. Hard to replicate results 

# Snakemake Workflow
Lets automate this process through the use of a snakemake workflow! Snakemake looks for a a file (called a `snakefile`) located either in the root directory or the `workflow` directory. This file has already been created at `workflow/snakefile`

## 1. Create a "target" rule
All snakemake files should have a master rule (called the target) which represents the end goal of the workflow. In this case it is a chart of the capacity. 

```python
rule all:
    input:
        "results/scenario/AnnualCapacity.png"
```

## 2. Create the rules to execute the scripts 
Rememmber, that the variable cost script must be run after the demand script! 

```python
rule capital_cost:
    input:
        "resources/data/AnnualDemand.csv"
    output:
        "results/scenario/data/AnnualDemand.csv"
    params:
        technology = "SPV",
        scaling_factor = 1.5
    shell: 
        "python workflow/scripts/capital_costs.py {input} {output} {params.technology} {params.scaling_factor}"
```

```python
rule emission_penalty:
    input:
        "resources/data/EmissionsPenalty.csv"
    output:
        "results/scenario/data/EmissionsPenalty.csv"
    params:
        start = 0,
        end = 100
    shell: 
        "python workflow/scripts/emission_penalty.py {input[0]} {output[0]} {params.start} {params.end}"
```

```python
rule demand:
    input:
        "resources/data/AnnualDemand.csv"
    output:
        "results/scenario/data/AnnualDemand.csv"
    params:
        scaling_factor = 2,
    shell: 
        "python workflow/scripts/demand.py {input} {output} {params.scaling_factor}"
```

```python
rule variable_cost:
    input:
        var_cost = "resources/data/VariableCosts.csv",
        demand = "results/scenario/data/AnnualDemand.csv"
    output:
        "results/scenario/data/VariableCosts.csv"
    shell: 
        "python workflow/scripts/variable_costs.py {input.demand} {input.var_cost} {output}"
```

## 3. Create the remaining files
In general, Snakemake rule outputs must be unique, meaning that the same file shouldn't be created through multiple rules. Therefore, we need to be careful not to create a rule that also outputs the files `AnnualDemand.csv`, `CapitalCosts.csv`, `EmissionsPenalty.csv`, and `VariableCosts.csv`.

We first create constants of the files we need: 
```python
OSEMOSYS_CSVS = os.listdir("resources/data")
CSVS_TO_CREATE = [
    "AnnualDemand.csv",
    "EmissionsPenalty.csv",
    "VariableCosts.csv",
    "CapitalCosts.csv"
]
CSVS_TO_COPY = [f for f in OSEMOSYS_CSVS if f not in CSVS_TO_CREATE]
```

And create a rule to only copy over the files in `CSVS_TO_COPY`:
```python 
rule copy_csv_files:
    input:
        expand("resources/data/{csv}", csv=OSEMOSYS_CSVS)
    output:
        expand("results/scenario/data/{csv}", csv=CSVS_TO_COPY)
    params:
        folder = directory("results/scenario/data")
    run:
        for path in input:
            _, f = os.path.split(path) # f will be in form of "file.csv"
            if f in CSVS_TO_CREATE:
                continue
            shutil.copy(path, os.path.join(params.folder, f))
```

## 4. Create the datafile rule 
```python 
rule otoole:
    input:
        expand("results/scenario/data/{csv}", csv=OSEMOSYS_CSVS)
    output:
        "results/scenario/data.txt"
    params:
        csv_dir = "results/scenario/data",
        config="resources/config.yaml"
    shell:
        "otoole convert csv datafile {params.csv_dir} {output} {params.config}"
```

## 5. Solve the model 
```python 
rule solve:
    input: 
        "results/scenario/data.txt"
    output:
        "results/scenario/results/TotalCapacityAnnual.csv"
    params:
        model="resources/osemosys.txt"
    shell:
        # since OSeMOSYS using GLPK requires a results directory from the 
        # place of running, we change the working directory to the location of the scenario
        """
        FILE="resources/data.txt" &&
        f="$(basename -- $FILE)" &&
        cd results/scenario &&
        glpsol -m ../../{params.model} -d $f
        """
```

## 6. Plot Results
```python
rule plot:
    input:
        "results/scenario/results/TotalCapacityAnnual.csv"
    output:
        "results/scenario/AnnualCapacity.png"
    shell:
        "python workflow/scripts/plot_capacity.py {input} {output} 'Scenario'"
```

## 7. Create missing directories 
```python
rule create_scenario_dir:
    output:
        directory("results/scenario/data")
    shell:
        "mkdir output"
```

```python
rule results_dir:
    input:
        "results/scenario"
    output:
        directory("results/scenario/results")
    shell:
        "mkdir {output}"
```

# Visualize the workflow 
Before executing lets visualize the workflow
```bash
snakemake --dag all | dot -Tpdf > dag.pdf
```

# Execute the workflow 
The `-c` flag sets the number of cores to use in the workflow. If no value is given (as shown below), snakemake will use all availbale cores. If you only wanted 2 cores to be used, you would set the flag as `-c2`. 
```bash 
snakemake -c
```

```bash
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
all                     1              1              1
capital_cost            1              1              1
copy_csv_files          1              1              1
demand                  1              1              1
emission_penalty        1              1              1
otoole                  1              1              1
plot                    1              1              1
solve                   1              1              1
variable_cost           1              1              1
total                   9              1              1
```

# Issues 
We are still hardcoding in variable values. This is a reproduciable workflow, but not a flexible workflow. Lets fix that with wildcards and a configuration file! 

# Wildcards
Usually, it is useful to generalize a rule to be applicable to a number of e.g. datasets. For this purpose, wildcards can be used. Automatically resolved multiple named wildcards are a key feature and strength of Snakemake in comparison to other systems. ([Source](https://snakemake.readthedocs.io/en/stable/snakefiles/rules.html#wildcards))

Note that the target rule can not include wildcards. We need to explicitly tell Snakemake what files we want to create in the target rule. 

## 1. Change our Scenario name to a wildcard
**DO NOT CHANGE THE TARGET RULE TO INCLUDE WILDCARDS**
```python
rule capital_cost:
    input:
        "resources/data/CapitalCost.csv"
    output:
        "results/{scenario}/data/CapitalCost.csv"
    params:
        technology = "SPV",
        scaling_factor = 1.5
    shell: 
        "python workflow/scripts/capital_costs.py {input} {output} {params.technology} {params.scaling_factor}"
```

```python
rule otoole:
    input:
        expand("results/{{scenario}}/data/{csv}", csv=OSEMOSYS_CSVS)
    output:
        "results/{scenario}/data.txt"
    params:
        csv_dir = "results/{scenario}/data",
        config="resources/config.yaml"
    shell:
        "otoole convert csv datafile {params.csv_dir} {output} {params.config}"
```

```python
rule plot:
    input:
        "results/{scenario}/results/TotalCapacityAnnual.csv"
    output:
        "results/{scenario}/AnnualCapacity.png"
    shell:
        "python workflow/scripts/plot_capacity.py {input} {output} {wildcards.scenario}"
```

## 2. Hard code in scenarios
```python
SCENARIOS = ["Kamaria", "Teddy", "Pierre", "Narges", "Yalda", "Trevor", "Elias"]

rule all:
    input:
        expand("results/{scenario}/AnnualCapacity.png", scenario=SCENARIOS)
```

## 3. Visualize the workflow 
```bash
snakemake --dag all | dot -Tpdf > dag.pdf
```

## 4. Run the workflow 
```bash 
snakemake -c
```

```bash
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
all                     1              1              1
capital_cost            4              1              1
copy_csv_files          4              1              1
demand                  4              1              1
emission_penalty        4              1              1
otoole                  4              1              1
plot                    4              1              1
solve                   4              1              1
variable_cost           4              1              1
total                  33              1              1
```

# Configuration File 
We have now introduced flexibility and modularity into our workflow, but it still requires us to hard code in values for the scenario name, scaling factor, ect. Lets move all this information to a configuration file! 

## Configuartion Setup 
First, set up a `config/config.yaml` file

```yaml
scenarios:
  scenario_one:
    capex:
      tech: "SPV"
      scale: 1.5
    emission_penalty:
      start: 0
      end: 100
    demand:
      scale: 2
  scenario_two:
    ...
```

## Import into workflow 
Snakemake will autmatically parse this out as a dictionary with the variable `config`
```python
configfile: "config/config.yaml"
```

## Update workflow 
You can not directly evaluate expressions in `input`, `output`, `params` sections. Therefore, we
use input functions.

```python
rule capital_cost:
    input:
        "resources/data/CapitalCost.csv"
    output:
        "results/{scenario}/data/CapitalCost.csv"
    params:
        technology = lambda wildcards: config["scenarios"][wildcards.scenario]["capex"]["tech"],
        scaling_factor = lambda wildcards: config["scenarios"][wildcards.scenario]["capex"]["scale"],
    shell: 
        "python workflow/scripts/capital_costs.py {input} {output} {params.technology} {params.scaling_factor}"
```

```python 
rule emission_penalty:
    input:
        "resources/data/EmissionsPenalty.csv"
    output:
        "results/{scenario}/data/EmissionsPenalty.csv"
    params:
        start = lambda wildcards: config["scenarios"][wildcards.scenario]["emission_penalty"]["start"],
        end = lambda wildcards: config["scenarios"][wildcards.scenario]["emission_penalty"]["start"],
    shell: 
        "python workflow/scripts/emission_penalty.py {input} {output} {params.start} {params.end}"
```

```python 
rule demand:
    input:
        "resources/data/SpecifiedAnnualDemand.csv"
    output:
        "results/{scenario}/data/SpecifiedAnnualDemand.csv"
    params:
        scaling_factor = lambda wildcards: config["scenarios"][wildcards.scenario]["demand"]["scale"],
    shell: 
        "python workflow/scripts/demand.py {input} {output} {params.scaling_factor}"
```

# More Wildcards! 
What happens if we want to itterate over LOTS of different parameter values? We could 
implement logic at the start of the script to generate many different permutations of 
values. OR we can just introduce more wildcards! 

Lets generalize all our parameter values and use them to build our scenario names. Our sceanrios
will follow the structure: 
`d{scale_factor}/capex_{tech}{scale}/ep{start}/ep{end}/`

## Configuration File
```yaml
scenarios:
  capex:
    techs: ["SPV", "HYD", "GAS"]
    scale: [0.25, 4]
  emission_penalty:
    start: [0, 25]
    end: [25, 100]
  demand:
    start: 0.5
    end: 4.5
    step: 1
```

## Workflow Update


## Visualize Workflow 


## Run Workflow 
```bash
snakemake -c
```

```yaml
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
all                     1              1              1
capital_cost          360              1              1
copy_csv_files        360              1              1
demand                360              1              1
emission_penalty      360              1              1
otoole                360              1              1
plot                  360              1              1
solve                 360              1              1
variable_cost         360              1              1
total                2881              1              1

Select jobs to execute...

[Fri Mar 31 17:46:20 2023]
rule capital_cost:
    input: resources/data/CapitalCost.csv
    output: results/d2.5/capex_SPV2/ep50/ep100/data/CapitalCost.csv
    jobid: 1526
    reason: Missing output files: results/d2.5/capex_SPV2/ep50/ep100/data/CapitalCost.csv
    wildcards: d_scale=2.5, tech=SPV, capex_scale=2, ep_start=50, ep_end=100
    resources: tmpdir=/tmp
```

# Other Notes
- Snakemake scales easy to cloud infrastructure
- Lots of other functionality (input functions, docker, flags, rule order, temporary files, modularity, Jupyter integration, wrapers, remote files, ect...)
- Create scenarios from csv files rather than yaml files 
- Powerful for uncertanity analysis, sensitivity analysis, and scenario analysis functions 
- Only reruns necessary rules 
- Very easy way to make your work accessible 